In [1]:
# Cell 0：环境 & 路径（之前已经跑通的，保留）

import os
import pandas as pd
import numpy as np

RAW_DIR = r"D:\ai_env\工业AI练手项目1：园区 站点空气异常审计\data\raw"
OUT_DIR = r"D:\ai_env\工业AI练手项目1：园区 站点空气异常审计\data\processed"

os.makedirs(OUT_DIR, exist_ok=True)


In [2]:
# Cell 1：加载工业区站点清单（一次性）

industrial_sites = pd.read_csv(
    os.path.join(OUT_DIR, "houston_industrial_sites.csv")
)

industrial_site_ids = set(industrial_sites["site_id"].astype(str))

len(industrial_site_ids)


114

In [3]:
# Cell 2：只处理一个污染物（先用 SO₂）

so2_path = os.path.join(RAW_DIR, "(二氧化硫SO2)hourly_42401_2024.csv")


In [4]:
# 新Cell 3：用 chunksize 读取 + 站点过滤（核心）工业区 SO₂ 过滤（完全匹配列名）


so2_path = os.path.join(RAW_DIR, "(二氧化硫SO2)hourly_42401_2024.csv")

chunks = []

use_cols = [
    "State Code",
    "County Code",
    "Site Num",
    "Date Local",
    "Time Local",
    "Sample Measurement",
    "Units of Measure"
]

for chunk in pd.read_csv(so2_path, usecols=use_cols, chunksize=200_000):
    
    chunk["site_id"] = (
        chunk["State Code"].astype(str).str.zfill(2) + "-"
        + chunk["County Code"].astype(str).str.zfill(3) + "-"
        + chunk["Site Num"].astype(str).str.zfill(4)
    )
    
    filtered = chunk[chunk["site_id"].isin(industrial_site_ids)]
    
    if not filtered.empty:
        chunks.append(filtered)

so2_industrial = pd.concat(chunks, ignore_index=True)

so2_industrial.shape


(32066, 8)

In [5]:
# Cell 4：时间字段整理（后面一切的基础）

# Cell 4 备选修正（仅改列名）

# 1. 合并日期+时间为标准时间戳（核心）

so2_industrial["datetime"] = pd.to_datetime(
    so2_industrial["Date Local"] + " " + so2_industrial["Time Local"],  # 用原始列名  拼接"日期 时间"字符串（如"2024-01-01 00:00"）
    errors="coerce"            # 遇到无法解析的时间（如"2024-02-30 25:00"），不报错，转为NaT（空时间）
)
# 2. 按站点+时间排序（保证时序逻辑）
so2_industrial = so2_industrial.sort_values(
    ["site_id", "datetime"] # 先按站点ID分组，再按时间升序排列
)
# 3. 预览前5行数据（仅展示，不改变数据本身）
so2_industrial.head() 


,State Code,County Code,Site Num,Date Local,Time Local,Sample Measurement,Units of Measure,site_id,datetime
0,48,201,51,2024-01-01,00:00,0.2,Parts per billion,48-201-0051,2024-01-01 00:00:00
1,48,201,51,2024-01-01,01:00,0.4,Parts per billion,48-201-0051,2024-01-01 01:00:00
2,48,201,51,2024-01-01,02:00,0.0,Parts per billion,48-201-0051,2024-01-01 02:00:00
3,48,201,51,2024-01-01,03:00,-0.1,Parts per billion,48-201-0051,2024-01-01 03:00:00
4,48,201,51,2024-01-01,04:00,-0.1,Parts per billion,48-201-0051,2024-01-01 04:00:00


In [6]:
# Cell 5

print("unique sites in SO2 data:",
      so2_industrial["site_id"].nunique())


unique sites in SO2 data: 4


In [7]:
# 查看4个有数据的站点ID 确认这 4 个站点的具体 ID，以及是否在 114 个目标站点清单里
valid_site_ids = so2_industrial["site_id"].unique()
print("有SO₂数据的4个站点ID：", valid_site_ids)

# 验证这些ID是否在114个目标站点清单里
in_target = [sid in industrial_site_ids for sid in valid_site_ids]
print("是否属于目标站点：", in_target)  # 正常应全为True

有SO₂数据的4个站点ID： ['48-201-0051' '48-201-0416' '48-201-1035' '48-201-1039']
是否属于目标站点： [True, True, True, True]


In [8]:
# 正式审计环节 1 峰值检查

# 1. 确保时间排序

so2_industrial["datetime"] = pd.to_datetime(
    so2_industrial["Date Local"] + " " + so2_industrial["Time Local"],
    errors="coerce"
)

so2_industrial = so2_industrial.sort_values(
    ["site_id", "datetime"]
)

# 2. 计算每个站点 99.5 分位数阈值

thresholds = (
    so2_industrial
    .groupby("site_id")["Sample Measurement"]
    .quantile(0.995)
    .reset_index()
    .rename(columns={"Sample Measurement": "spike_threshold"})
)

# 3. 合并阈值

so2_industrial = so2_industrial.merge(
    thresholds,
    on="site_id",
    how="left"
)

# 4. 标记异常

so2_industrial["is_spike"] = (
    so2_industrial["Sample Measurement"]
    > so2_industrial["spike_threshold"]
)

# 5. 查看异常数量

print("Spike count:",
      so2_industrial["is_spike"].sum())


Spike count: 150


In [9]:
# 计算相邻小时差值 Δ异常（突变检测）

so2_industrial["delta"] = (
    so2_industrial
    .groupby("site_id")["Sample Measurement"]
    .diff()
)

# 计算每个站点 delta 的 99.5 分位数

delta_thresholds = (
    so2_industrial
    .groupby("site_id")["delta"]
    .quantile(0.995)
    .reset_index()
    .rename(columns={"delta": "delta_threshold"})
)

# 合并阈值
so2_industrial = so2_industrial.merge(
    delta_thresholds,
    on="site_id",
    how="left"
)

# 标记突变异常
so2_industrial["is_delta_spike"] = (
    so2_industrial["delta"]
    > so2_industrial["delta_threshold"]
)

print("Delta spike count:",
      so2_industrial["is_delta_spike"].sum())


Delta spike count: 156


In [10]:
# 真正有价值的一步：连续异常簇检测
# 只保留 delta spike 行
delta_events = so2_industrial[
    so2_industrial["is_delta_spike"]
].copy()

# 标记连续时间段（按站点）

delta_events["event_group"] = (
    delta_events
    .groupby("site_id")["datetime"]
    .diff()
    .dt.total_seconds()
    .div(3600)
    .ne(1)  # 如果不是连续1小时
    .cumsum()
)

# 计算每个 group 的长度
event_summary = (
    delta_events
    .groupby(["site_id", "event_group"])
    .size()
    .reset_index(name="duration_hours")
)

# 找出持续 >=3 小时的异常
long_events = event_summary[
    event_summary["duration_hours"] >= 3
]

long_events


,site_id,event_group,duration_hours


In [11]:
# Cell 复制so2 的逻辑，审计 no2 ，这步按站点筛选数据（需要先运行cell 0-1）

no2_path = os.path.join(RAW_DIR, "(二氧化氮NO2)hourly_42602_2024.csv")

chunks = []

use_cols = [
    "State Code",
    "County Code",
    "Site Num",
    "Date Local",
    "Time Local",
    "Sample Measurement"
]

for chunk in pd.read_csv(no2_path, usecols=use_cols, chunksize=200_000):

    chunk["site_id"] = (
        chunk["State Code"].astype(str).str.zfill(2) + "-"
        + chunk["County Code"].astype(str).str.zfill(3) + "-"
        + chunk["Site Num"].astype(str).str.zfill(4)
    )

    filtered = chunk[chunk["site_id"].isin(industrial_site_ids)]

    if not filtered.empty:
        chunks.append(filtered)

no2_industrial = pd.concat(chunks, ignore_index=True)

print("NO2 shape:", no2_industrial.shape)
print("NO2 unique sites:", no2_industrial["site_id"].nunique())


NO2 shape: (102732, 7)
NO2 unique sites: 13


In [12]:
# 给 NO₂ 做 spike + delta（一次到位）峰值和变化
# 时间字段

no2_industrial["datetime"] = pd.to_datetime(
    no2_industrial["Date Local"] + " " + no2_industrial["Time Local"],
    errors="coerce"
)

no2_industrial = no2_industrial.sort_values(
    ["site_id", "datetime"]
)

# ---------- Spike ----------

no2_thresholds = (
    no2_industrial
    .groupby("site_id")["Sample Measurement"]
    .quantile(0.995)
    .reset_index()
    .rename(columns={"Sample Measurement": "spike_threshold"})
)

no2_industrial = no2_industrial.merge(
    no2_thresholds,
    on="site_id",
    how="left"
)

no2_industrial["is_spike"] = (
    no2_industrial["Sample Measurement"]
    > no2_industrial["spike_threshold"]
)

# ---------- Delta ----------

no2_industrial["delta"] = (
    no2_industrial
    .groupby("site_id")["Sample Measurement"]
    .diff()
)

delta_thresholds_no2 = (
    no2_industrial
    .groupby("site_id")["delta"]
    .quantile(0.995)
    .reset_index()
    .rename(columns={"delta": "delta_threshold"})
)

no2_industrial = no2_industrial.merge(
    delta_thresholds_no2,
    on="site_id",
    how="left"
)

no2_industrial["is_delta_spike"] = (
    no2_industrial["delta"]
    > no2_industrial["delta_threshold"]
)

print("NO2 spike count:", no2_industrial["is_spike"].sum())
print("NO2 delta spike count:", no2_industrial["is_delta_spike"].sum())


NO2 spike count: 515
NO2 delta spike count: 514


In [13]:
# 构建精简异常表
# 只保留必要字段

so2_spikes = so2_industrial[
    ["site_id", "datetime", "is_spike"]
].rename(columns={"is_spike": "so2_spike"})

no2_spikes = no2_industrial[
    ["site_id", "datetime", "is_spike"]
].rename(columns={"is_spike": "no2_spike"})

# merge
merged = pd.merge(
    so2_spikes,
    no2_spikes,
    on=["site_id", "datetime"],
    how="inner"
)

print("Merged rows:", merged.shape)


Merged rows: (21953, 4)


In [14]:
# 计算联动异常

merged["joint_spike"] = (
    merged["so2_spike"] & merged["no2_spike"]
)

print("Joint spike count:", merged["joint_spike"].sum())


Joint spike count: 5


In [15]:
#-------- 站点级风险指标 产品化-----------

In [16]:
# Step 1 计算每个站点的 spike 率

#  cell 1 二氧化硫 SO₂站点统计

so2_stats = (
    so2_industrial
    .groupby("site_id")
    .agg(
        so2_hours=("datetime", "count"),
        so2_spikes=("is_spike", "sum")
    )
)

so2_stats["so2_spike_rate"] = (
    so2_stats["so2_spikes"] / so2_stats["so2_hours"]
)

so2_stats.head()


,so2_hours,so2_spikes,so2_spike_rate
site_id,,,
48-201-0051,8454,40,0.004731
48-201-0416,8236,40,0.004857
48-201-1035,8298,35,0.004218
48-201-1039,7078,35,0.004945


In [17]:
# cell 2 NO₂站点统计

no2_stats = (
    no2_industrial
    .groupby("site_id")
    .agg(
        no2_hours=("datetime", "count"),
        no2_spikes=("is_spike", "sum")
    )
)

no2_stats["no2_spike_rate"] = (
    no2_stats["no2_spikes"] / no2_stats["no2_hours"]
)

no2_stats.head()


,no2_hours,no2_spikes,no2_spike_rate
site_id,,,
48-201-0024,7237,36,0.004974
48-201-0026,8369,42,0.005019
48-201-0047,6787,34,0.005010
48-201-0055,8150,40,0.004908
48-201-0416,8248,42,0.005092


In [18]:
# Step 2 — 计算联动 spike 率
# cell 3

joint_stats = (
    merged
    .groupby("site_id")
    .agg(
        joint_hours=("datetime", "count"),
        joint_spikes=("joint_spike", "sum")
    )
)

joint_stats["joint_spike_rate"] = (
    joint_stats["joint_spikes"] / joint_stats["joint_hours"]
)

joint_stats.head()


,joint_hours,joint_spikes,joint_spike_rate
site_id,,,
48-201-0416,7967,2,0.000251
48-201-1035,7053,0,0.000000
48-201-1039,6933,3,0.000433


In [19]:
# Step 3 — 合并成站点总表
# Cell 4

risk_table = (
    so2_stats
    .join(no2_stats, how="outer")
    .join(joint_stats, how="outer")
)

risk_table = risk_table.fillna(0)

print(risk_table.shape)
risk_table.head()


(14, 9)


,so2_hours,so2_spikes,so2_spike_rate,no2_hours,no2_spikes,no2_spike_rate,joint_hours,joint_spikes,joint_spike_rate
site_id,,,,,,,,,
48-201-0024,0.0,0.0,0.000000,7237.0,36.0,0.004974,0.0,0.0,0.0
48-201-0026,0.0,0.0,0.000000,8369.0,42.0,0.005019,0.0,0.0,0.0
48-201-0047,0.0,0.0,0.000000,6787.0,34.0,0.005010,0.0,0.0,0.0
48-201-0051,8454.0,40.0,0.004731,0.0,0.0,0.000000,0.0,0.0,0.0
48-201-0055,0.0,0.0,0.000000,8150.0,40.0,0.004908,0.0,0.0,0.0


In [20]:
# Step 4 — 标准化（关键） spike_rate 都很小 必须拉到可比较尺度
# Cell 5
for col in [
    "so2_spike_rate",
    "no2_spike_rate",
    "joint_spike_rate"
]:
    max_val = risk_table[col].max()
    if max_val > 0:
        risk_table[col + "_norm"] = risk_table[col] / max_val
    else:
        risk_table[col + "_norm"] = 0



In [21]:
# Step 5 — 计算 Risk Score（原型）

# 先用简单权重：SO₂：0.3 NO₂：0.3 联动：0.4（联动最重要）
# Cell 6
risk_table["risk_score"] = (
    0.3 * risk_table["so2_spike_rate_norm"] +
    0.3 * risk_table["no2_spike_rate_norm"] +
    0.4 * risk_table["joint_spike_rate_norm"]
)

risk_table = risk_table.sort_values(
    "risk_score", ascending=False
)

risk_table.head(10)


,so2_hours,so2_spikes,so2_spike_rate,no2_hours,no2_spikes,no2_spike_rate,joint_hours,joint_spikes,joint_spike_rate,so2_spike_rate_norm,no2_spike_rate_norm,joint_spike_rate_norm,risk_score
site_id,,,,,,,,,,,,,
48-201-1039,7078.0,35.0,0.004945,8467.0,42.0,0.004960,6933.0,3.0,0.000433,1.000000,0.974135,1.000000,0.992240
48-201-0416,8236.0,40.0,0.004857,8248.0,42.0,0.005092,7967.0,2.0,0.000251,0.982169,1.000000,0.580143,0.826708
48-201-1035,8298.0,35.0,0.004218,7197.0,36.0,0.005002,7053.0,0.0,0.000000,0.852977,0.982314,0.000000,0.550587
48-201-1015,0.0,0.0,0.000000,8073.0,41.0,0.005079,0.0,0.0,0.000000,0.000000,0.997352,0.000000,0.299205
48-201-0417,0.0,0.0,0.000000,8075.0,41.0,0.005077,0.0,0.0,0.000000,0.000000,0.997105,0.000000,0.299131
48-201-1034,0.0,0.0,0.000000,8096.0,41.0,0.005064,0.0,0.0,0.000000,0.000000,0.994518,0.000000,0.298355
48-201-1050,0.0,0.0,0.000000,7712.0,39.0,0.005057,0.0,0.0,0.000000,0.000000,0.993109,0.000000,0.297933
48-201-1052,0.0,0.0,0.000000,7964.0,40.0,0.005023,0.0,0.0,0.000000,0.000000,0.986343,0.000000,0.295903
48-201-0026,0.0,0.0,0.000000,8369.0,42.0,0.005019,0.0,0.0,0.000000,0.000000,0.985542,0.000000,0.295663


In [22]:
# Step 6 — 输出结果（客户视角）

# cell 7 

out_path = os.path.join(OUT_DIR, "houston_site_risk_scores.csv")
risk_table.to_csv(out_path)

print("Saved:", out_path)


Saved: D:\ai_env\工业AI练手项目1：园区 站点空气异常审计\data\processed\houston_site_risk_scores.csv


In [23]:
# 代码：生成 PDF 报告 跑完会得到：industrial_risk_report.pdf

# risk_table = 现在那个DataFrame

risk_table = risk_table.reset_index()


from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

doc = SimpleDocTemplate("industrial_risk_report.pdf", pagesize=A4)
styles = getSampleStyleSheet()
elements = []

# Title
elements.append(Paragraph("Industrial Air Risk Report", styles['Title']))
elements.append(Spacer(1,12))

# Summary
total_sites = len(risk_table)
top_site = risk_table.sort_values("risk_score", ascending=False).iloc[0]

summary = f"""
Total sites analyzed: {total_sites}<br/>
Highest risk site: {top_site.site_id}<br/>
Risk score: {top_site.risk_score:.3f}<br/><br/>
"""

elements.append(Paragraph(summary, styles['Normal']))
elements.append(Spacer(1,12))

# Ranking table
rt = risk_table.sort_values("risk_score", ascending=False)

table_data = [["Rank","Site","Risk","SO2 spikes","NO2 spikes","Joint spikes"]]

for i,row in enumerate(rt.itertuples(),1):
    table_data.append([
        i,
        row.site_id,
        round(row.risk_score,3),
        row.so2_spikes,
        row.no2_spikes,
        row.joint_spikes
    ])

table = Table(table_data)
table.setStyle([
    ('BACKGROUND',(0,0),(-1,0),colors.grey),
    ('GRID',(0,0),(-1,-1),1,colors.black)
])

elements.append(table)
elements.append(Spacer(1,24))

# High risk explanations
for row in rt.head(3).itertuples():

    txt=f"""
    <b>Site {row.site_id}</b><br/>
    Risk Score: {row.risk_score:.3f}<br/>
    SO2 spikes: {row.so2_spikes}<br/>
    NO2 spikes: {row.no2_spikes}<br/>
    Joint spikes: {row.joint_spikes}<br/><br/>

    Multi-pollutant abnormal events detected.<br/>
    Recommend priority inspection.<br/><br/>
    """

    elements.append(Paragraph(txt, styles['Normal']))
    elements.append(Spacer(1,12))

doc.build(elements)

print("PDF generated")


PDF generated


In [24]:
# 先打印列名，确认正确的列名是什么
print("risk_table的列名：", risk_table.columns.tolist())

risk_table的列名： ['site_id', 'so2_hours', 'so2_spikes', 'so2_spike_rate', 'no2_hours', 'no2_spikes', 'no2_spike_rate', 'joint_hours', 'joint_spikes', 'joint_spike_rate', 'so2_spike_rate_norm', 'no2_spike_rate_norm', 'joint_spike_rate_norm', 'risk_score']


In [25]:
#  替换的最终版 PDF 生成代码。 升级工业报告

# ========= Industrial-grade PDF Report =========

from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table
from reportlab.platypus import TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
from reportlab.pdfbase import pdfmetrics
from reportlab.lib.units import inch

import os

# ---- 输出路径（确保在项目里）
OUTPUT_PATH = os.path.join(os.getcwd(), "Industrial_Air_Risk_Report.pdf")

# ---- 注册中文/Unicode字体（防止字符问题）
pdfmetrics.registerFont(UnicodeCIDFont("HYSMyeongJo-Medium"))

styles = getSampleStyleSheet()
styles["Normal"].fontName = "HYSMyeongJo-Medium"
styles["Title"].fontName = "HYSMyeongJo-Medium"

doc = SimpleDocTemplate(
    OUTPUT_PATH,
    pagesize=A4,
)

elements = []

# =========================
# 1️⃣ Title
# =========================
elements.append(Paragraph("Industrial Emission Risk Screening Report", styles['Title']))
elements.append(Spacer(1, 12))

elements.append(Paragraph(
"Houston Industrial Area — EPA Monitoring Data (2024)",
styles['Normal']))
elements.append(Spacer(1, 20))


# =========================
# 2️⃣ Executive Summary
# =========================

total_sites = len(risk_table)

top_site_row = risk_table.sort_values("risk_score", ascending=False).reset_index().iloc[0]
top_site_id = top_site_row["site_id"]
top_score = top_site_row["risk_score"]

summary_text = f"""
<b>Executive Summary</b><br/><br/>

Total industrial monitoring sites analyzed: {total_sites}<br/>
Highest risk monitoring site: {top_site_id}<br/>
Risk score: {top_score:.3f}<br/><br/>

Multi-pollutant abnormal emission patterns were detected across several monitoring locations.
The highest-ranked site shows repeated SO2 and NO2 spikes, including coincident events,
which may indicate combustion-related industrial emission activity.<br/><br/>

Recommendation: prioritize inspection or operational review near the identified location.
"""

elements.append(Paragraph(summary_text, styles['Normal']))
elements.append(Spacer(1, 20))


# =========================
# 3️⃣ Ranking Table
# =========================

table_df = (
    risk_table
    .reset_index()
    .sort_values("risk_score", ascending=False)
)

display_cols = [
    "site_id",
    "risk_score",
    "so2_spikes",
    "no2_spikes",
    "joint_spikes"
]

table_data = [["Rank","Site","Risk","SO2 spikes","NO2 spikes","Joint spikes"]]

for i, row in table_df.iterrows():
    table_data.append([
        len(table_data),
        row["site_id"],
        f"{row['risk_score']:.3f}",
        int(row["so2_spikes"]),
        int(row["no2_spikes"]),
        int(row["joint_spikes"]),
    ])

tbl = Table(table_data)

tbl.setStyle(TableStyle([
    ('GRID',(0,0),(-1,-1),0.5,colors.black),
    ('BACKGROUND',(0,0),(-1,0),colors.lightgrey),
]))

elements.append(tbl)
elements.append(Spacer(1, 20))


# =========================
# 4️⃣ Site Risk Explanations
# =========================

elements.append(Paragraph("<b>Site Risk Interpretation</b>", styles['Normal']))
elements.append(Spacer(1, 10))

top_sites = table_df.head(5)

for _, row in top_sites.iterrows():

    site = row["site_id"]
    score = row["risk_score"]
    so2 = int(row["so2_spikes"])
    no2 = int(row["no2_spikes"])
    joint = int(row["joint_spikes"])

    text = f"""
<b>Site {site}</b><br/>
Risk score: {score:.3f}<br/>
SO2 abnormal events: {so2}<br/>
NO2 abnormal events: {no2}<br/>
Coincident multi-pollutant events: {joint}<br/><br/>

Interpretation:<br/>
Frequent pollutant spikes detected. Pattern consistent with possible industrial combustion emissions,
refinery activity, or abnormal flaring behavior. Multi-pollutant coincidence increases likelihood of
real emission events rather than sensor noise.<br/><br/>

Recommendation: targeted inspection or emissions audit suggested.<br/><br/>
"""

    elements.append(Paragraph(text, styles['Normal']))
    elements.append(Spacer(1,12))


# =========================
# 5️⃣ Build PDF
# =========================

doc.build(elements)

print("PDF generated at:")
print(OUTPUT_PATH)


PDF generated at:
d:\ai_env\工业AI练手项目1 最小闭环\Industrial_Air_Risk_Report.pdf


In [29]:
#  生成地图代码（直接运行）

import folium
import pandas as pd
import os

# ========================
# 1️⃣ 读取站点坐标
# ========================

# sites = pd.read_csv(r"data/raw/(站点列表)aqs_sites.csv")

sites = pd.read_csv(r"D:\ai_env\工业AI练手项目1：园区 站点空气异常审计\data\raw\(站点列表)aqs_sites.csv")

sites["site_id"] = (
    sites["State Code"].astype(str).str.zfill(2) + "-" +
    sites["County Code"].astype(str).str.zfill(3) + "-" +
    sites["Site Number"].astype(str).str.zfill(4)
)

sites = sites[["site_id","Latitude","Longitude"]]


# ========================
# 2️⃣ 合并风险表
# ========================

map_df = risk_table.reset_index().merge(
    sites,
    on="site_id",
    how="left"
)

map_df = map_df.dropna(subset=["Latitude","Longitude"])


# ========================
# 3️⃣ 创建地图（Houston中心）
# ========================

houston_center = [29.7604, -95.3698]

m = folium.Map(
    location=houston_center,
    zoom_start=10
)


# ========================
# 4️⃣ 风险颜色函数
# ========================

def risk_color(score):
    if score > 0.8:
        return "red"
    elif score > 0.5:
        return "orange"
    else:
        return "green"


# ========================
# 5️⃣ 添加站点
# ========================

for _, row in map_df.iterrows():

    popup = f"""
    Site: {row['site_id']}<br>
    Risk score: {row['risk_score']:.3f}<br>
    SO2 spikes: {int(row['so2_spikes'])}<br>
    NO2 spikes: {int(row['no2_spikes'])}<br>
    Joint spikes: {int(row['joint_spikes'])}
    """

    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=8,
        color=risk_color(row["risk_score"]),
        fill=True,
        fill_opacity=0.7,
        popup=popup
    ).add_to(m)


# ========================
# 6️⃣ 保存HTML
# ========================

OUTPUT_MAP = os.path.join(os.getcwd(), "Industrial_Risk_Map.html")

m.save(OUTPUT_MAP)

print("Map saved at:")
print(OUTPUT_MAP)


Map saved at:
d:\ai_env\工业AI练手项目1 最小闭环\Industrial_Risk_Map.html


In [27]:
import subprocess
import sys

# 安装folium、tqdm（之前代码用到的），精准安装到当前虚拟环境
try:
    # 执行pip安装命令，指向当前虚拟环境的pip
    subprocess.check_call(
        [sys.executable, "-m", "pip", "install", "folium", "tqdm", "pandas"],
        stdout=subprocess.PIPE,  # 隐藏冗余输出
        stderr=subprocess.PIPE
    )
    print("✅ folium等模块已成功安装到当前虚拟环境！")
except Exception as e:
    print(f"❌ 安装失败：{e}")

✅ folium等模块已成功安装到当前虚拟环境！


In [ ]:
# 时间轴污染动画地图

In [35]:
# 重建 SO2 / NO2 spike 数据（最小安全版）

import os
import pandas as pd

# 替换这几行即可，其他逻辑保留
PROJECT_ROOT = r"D:\ai_env\工业AI练手项目1：园区 站点空气异常审计"
processed_file_path = os.path.join(PROJECT_ROOT, "data", "processed", "houston_industrial_sites.csv")
industrial_sites = pd.read_csv(processed_file_path, encoding="utf-8")
industrial_site_ids = set(industrial_sites["site_id"])

SO2_FILE="(二氧化硫SO2)hourly_42401_2024.csv"
NO2_FILE="(二氧化氮NO2)hourly_42602_2024.csv"

def load_pollutant(file):

    path=os.path.join(RAW_DIR,file)

    usecols=[
        "State Code","County Code","Site Num",
        "Date Local","Time Local",
        "Sample Measurement"
    ]

    chunks=[]

    for chunk in pd.read_csv(path,usecols=usecols,chunksize=200000):

        chunk["site_id"]=(
            chunk["State Code"].astype(str).str.zfill(2)+"-"+
            chunk["County Code"].astype(str).str.zfill(3)+"-"+
            chunk["Site Num"].astype(str).str.zfill(4)
        )

        chunk=chunk[chunk["site_id"].isin(industrial_site_ids)]

        chunks.append(chunk)

    df=pd.concat(chunks,ignore_index=True)

    # spike定义（简单稳健）
    threshold=df["Sample Measurement"].quantile(0.99)

    df["spike"]=(df["Sample Measurement"]>threshold).astype(int)

    return df

so2_df=load_pollutant(SO2_FILE)
no2_df=load_pollutant(NO2_FILE)

print("SO2:",so2_df.shape)
print("NO2:",no2_df.shape)


SO2: (32066, 8)
NO2: (102732, 8)


In [36]:
# 需要 每个 spike 的具体时间 + 坐标。
# 准备 spike 明细表

# ============================
# 生成事件级 spike 表（SO2+NO2）
# ============================

import pandas as pd

# SO2 spikes
so2_events = so2_df[so2_df["spike"]==1].copy()
so2_events["pollutant"] = "SO2"

# NO2 spikes
no2_events = no2_df[no2_df["spike"]==1].copy()
no2_events["pollutant"] = "NO2"

events = pd.concat([so2_events,no2_events],ignore_index=True)

print("events:",events.shape)
events.head()


events: (1332, 9)


,State Code,County Code,Site Num,Date Local,Time Local,Sample Measurement,site_id,spike,pollutant
0,48,201,51,2024-01-18,17:00,2.9,48-201-0051,1,SO2
1,48,201,51,2024-01-18,18:00,3.8,48-201-0051,1,SO2
2,48,201,51,2024-05-13,11:00,5.9,48-201-0051,1,SO2
3,48,201,51,2024-05-27,08:00,4.1,48-201-0051,1,SO2
4,48,201,51,2024-05-27,09:00,3.9,48-201-0051,1,SO2


In [37]:
# 生成时间字段（动画必须）
events["datetime"]=pd.to_datetime(
    events["Date Local"]+" "+events["Time Local"]
)

events=events.sort_values("datetime")

events.head()


,State Code,County Code,Site Num,Date Local,Time Local,Sample Measurement,site_id,spike,pollutant,datetime
57,48,201,416,2024-01-02,10:00,2.4,48-201-0416,1,SO2,2024-01-02 10:00:00
58,48,201,416,2024-01-02,12:00,2.3,48-201-0416,1,SO2,2024-01-02 12:00:00
820,48,201,1052,2024-01-04,10:00,39.4,48-201-1052,1,NO2,2024-01-04 10:00:00
1200,48,201,1066,2024-01-05,18:00,39.9,48-201-1066,1,NO2,2024-01-05 18:00:00
598,48,201,1034,2024-01-05,18:00,41.1,48-201-1034,1,NO2,2024-01-05 18:00:00


In [39]:
# 限制数据量（否则动画太大）

# 先补全：假设你有站点经纬度表 sites（之前读取的aqs_sites.csv）
# 第一步：给events关联经纬度（通过site_id匹配）
events = events.merge(sites[["site_id", "Latitude", "Longitude"]], on="site_id", how="left")
events = events.dropna(subset=["Latitude", "Longitude"])  # 删掉无经纬度的行

# 限制数据量（否则动画太大）
events["date"] = events["datetime"].dt.date

# 第二步：重新groupby（此时有Latitude/Longitude列了）
daily_events = events.groupby(
    ["date","Latitude","Longitude","pollutant"]
).size().reset_index(name="count")

print(daily_events.shape)


(508, 5)


In [40]:
# 生成动画地图 HTML

import folium
from folium.plugins import TimestampedGeoJson
import json

features=[]

for _,row in daily_events.iterrows():

    features.append({
        "type":"Feature",
        "geometry":{
            "type":"Point",
            "coordinates":[row["Longitude"],row["Latitude"]],
        },
        "properties":{
            "time":str(row["date"]),
            "popup":f"{row['pollutant']} spikes:{row['count']}",
            "icon":"circle",
            "iconstyle":{
                "fillColor":"red" if row["pollutant"]=="SO2" else "blue",
                "fillOpacity":0.7,
                "stroke":"true",
                "radius":6
            }
        }
    })

m=folium.Map(location=[29.73,-95.30],zoom_start=10)

TimestampedGeoJson(
    {"type":"FeatureCollection","features":features},
    period="P1D",
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
).add_to(m)

m.save("houston_pollution_animation.html")

print("Saved: houston_pollution_animation.html")


Saved: houston_pollution_animation.html
